<a href="https://colab.research.google.com/github/VikaBilyk/3a-lab-OOOP/blob/main/lab2_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np

def customSign(x):
    return 1 if x > 0 else (-1 if x < 0 else 0)

def isSymmetric(matrix):
    return np.array_equal(matrix, matrix.T)

def determinant_from_square_root_method(dMatrix, sMatrix):
    n = dMatrix.shape[0]
    prod_d = 1  # Добуток елементів dMatrix
    prod_s = 1  # Добуток квадратів елементів sMatrix

    for i in range(n):
        prod_d *= dMatrix[i][i]  # Добуток елементів dMatrix
        prod_s *= sMatrix[i][i] ** 2  # Добуток квадратів елементів sMatrix

    detA = prod_d * prod_s

    return detA

def squareRootMethod(aMatrix, bVector, print_output=False):

    if not isSymmetric(aMatrix):
        raise ValueError("Матриця не є симетричною. Метод квадратного кореня можна застосовувати лише до симетричних матриць.")

    n = aMatrix.shape[0]
    dMatrix = np.zeros((n, n))
    sMatrix = np.zeros((n, n))

    # Обчислення елементів матриць D і S
    for i in range(n):
        sum_s = 0
        for p in range(i):
            sum_s += (sMatrix[p][i] ** 2) * dMatrix[p][p]

        # Обчислення d_ii за формулою
        dMatrix[i][i] = customSign(aMatrix[i][i] - sum_s)

        # Обчислення s_ii за формулою
        sMatrix[i][i] = np.sqrt(abs(aMatrix[i][i] - sum_s))

        # Обчислення s_ij для j > i
        for j in range(i + 1, n):
            sum_s = 0
            for p in range(i):
                sum_s += sMatrix[p][i] * dMatrix[p][p] * sMatrix[p][j]

            # Формула для s_ij
            sMatrix[i][j] = (aMatrix[i][j] - sum_s) / (dMatrix[i][i] * sMatrix[i][i])

    # Транспонування матриці S
    S_transpose = np.transpose(sMatrix)

    # Обчислення добутку S^T * D
    StD = np.dot(S_transpose, dMatrix)

    # Виведення результатів
    if print_output:
        print("Матриця A:")
        print(aMatrix)
        print("\nМатриця D:")
        print(dMatrix)
        print("\nМатриця S:")
        print(sMatrix)
        print("\nМатриця S^T:")
        print(S_transpose)
        print("\nМатриця S^T * D:")
        print(StD)

        # Визначник матриці A
        det_A = determinant_from_square_root_method(dMatrix, sMatrix)
        print("\nВизначник матриці A (detA):", det_A)

    # Пряма заміна для знаходження y
    y = solve_eq_mtrx(StD, bVector)

    # Зворотна заміна для знаходження x
    x = solve_eq_mtrx_inverse(sMatrix, y)

    # Пошук оберненої матриці
    A_inv = find_inverse_mtrx(StD, S_transpose)

    if print_output:
        print("\nОбернена матриця A:")
        print(A_inv)

        # Обчислення числа обумовленості
        cond_number = condition_number(aMatrix)
        print("\nЧисло обумовленості матриці A:", cond_number)

    return x, A_inv

def find_inverse_mtrx(R, S_t):
    Y = solve_inverse_mtrx(E, R)
    X = solve_inverse_mtrx(Y, S_t)
    A_inv = np.zeros((5, 5), dtype=float)
    transposed_matrix(X, A_inv)
    return A_inv

def solve_inverse_mtrx(E, R):
    y = np.zeros(5)
    Y = np.zeros((5, 5), dtype=float)
    for j in range(5):
        y = solve_eq_mtrx(R, E[j])
        for i in range(5):
            Y[i][j] = y[i]
    return Y

def solve_eq_mtrx(R, b):
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        sum = 0
        for j in range(i):
            sum += R[i][j] * y[j]
        y[i] = (b[i] - sum) / R[i][i]
    return y

def solve_eq_mtrx_inverse(R, b):
    n = len(b)
    y = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum = 0
        for j in range(i + 1, n):
            sum += R[i][j] * y[j]
        y[i] = (b[i] - sum) / R[i][i]
    return y

def transposed_matrix(S, S_transpose):
    n = S.shape[0]
    for i in range(n):
        for j in range(n):
            S_transpose[i][j] = S[j][i]
    return

def matrix_norm_inf(A):
    """Обчислення норми матриці за максимумом суми абсолютних значень елементів по рядках."""
    n = len(A)
    max_sum = 0
    for i in range(n):
        row_sum = 0
        for j in range(n):
            row_sum += abs(A[i][j])
        if row_sum > max_sum:
            max_sum = row_sum
    return max_sum

def condition_number(A):
    """Обчислює число обумовленості матриці A."""
    norm_A = matrix_norm_inf(A)
    inv_A = find_inverse_mtrx(A, A.T)
    norm_inv_A = matrix_norm_inf(inv_A)
    return norm_A * norm_inv_A

# Функція для обчислення детермінанту підматриці
def calculateDeterminant(matrix):
    n = len(matrix)
    det = 1
    A = [row[:] for row in matrix]

    for i in range(n):
        max_row = i + max(range(n - i), key=lambda k: abs(A[i + k][i]))
        if A[max_row][i] == 0:
            return 0

        if max_row != i:
            A[i], A[max_row] = A[max_row], A[i]
            det *= -1

        for j in range(i + 1, n):
            factor = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]

        det *= A[i][i]

    return det

# Функція для обчислення головних мінорів
def calculatePrincipalMinors(matrix):
    minors = []
    for k in range(1, len(matrix) + 1):
        submatrix = [row[:k] for row in matrix[:k]]
        det = calculateDeterminant(submatrix)  # Обчислюємо детермінант підматриці
        minors.append(det)
    return minors

def zeidelMethod(aMatrix, bVector, eps=1e-6):
    n = len(bVector)
    x = [0] * n
    x_new = x.copy()

    iteration = 0
    while True:
        iteration += 1
        for i in range(n):
            sum1 = sum(aMatrix[i][j] * x_new[j] for j in range(i))
            sum2 = sum(aMatrix[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (bVector[i] - sum1 - sum2) / aMatrix[i][i]

        # Обчислення евклідової норми
        diff = 0
        for i in range(n):
            diff += (x_new[i] - x[i]) ** 2

        # Якщо сума квадратів різниць менша за квадрат eps, то завершуємо
        if diff ** 0.5 < eps:
            return x_new

        x = x_new.copy()

# Використання однакової матриці та вектора для обох методів
aMatrix = np.array([[10, -1, 2, 0, 3],
                    [-1, 11, -1, 3, 1],
                    [2, -1, 10, -1, 1],
                    [0, 3, -1, 8, 2],
                    [3, 1, 1, 2, 10]])
bVector = np.array([6, 25, -11, 15, 20])

E = [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]

# Метод квадратного кореня з виводом матриць
roots_square, A_inv_square = squareRootMethod(aMatrix, bVector, print_output=True)
print("\nКорені (Метод квадратного кореня):", roots_square)
# print("\nОбернена матриця (Метод квадратного кореня):", A_inv_square)

# Метод Зейделя
roots_zeidel = zeidelMethod(aMatrix, bVector)
print("\nКорені (Метод Зейделя):", roots_zeidel)

# Обчислення і перевірка головних мінорів
principal_minors = calculatePrincipalMinors(aMatrix)
is_positive_definite = all(minor > 0 for minor in principal_minors)

if is_positive_definite:
    print("\nМатриця A є додатньо визначеною.")
else:
    print("\nМатриця A не є додатньо визначеною.")


Матриця A:
[[10 -1  2  0  3]
 [-1 11 -1  3  1]
 [ 2 -1 10 -1  1]
 [ 0  3 -1  8  2]
 [ 3  1  1  2 10]]

Матриця D:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Матриця S:
[[ 3.16227766 -0.31622777  0.63245553  0.          0.9486833 ]
 [ 0.          3.3015148  -0.24231301  0.9086738   0.39375865]
 [ 0.          0.          3.08889696 -0.25245792  0.16038503]
 [ 0.          0.          0.          2.6665665   0.63103332]
 [ 0.          0.          0.          0.          2.91907994]]

Матриця S^T:
[[ 3.16227766  0.          0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.          0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665   0.        ]
 [ 0.9486833   0.39375865  0.16038503  0.63103332  2.91907994]]

Матриця S^T * D:
[[ 3.16227766  0.          0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.          0.   